In [79]:
import numpy as np
from IPython.display import Audio


# Параметры звука
sample_rate = 44100  # Частота дискретизации (Гц)
dot_duration = 0.1    # Длительность точки (секунды)
dash_duration = 0.3   # Длительность тире (секунды)
pause_duration = 0.1  # Длительность паузы между сигналами (секунды)
word_pause = 0.5      # Длительность паузы между буквами (секунды)
frequency = 440       # Частота сигнала (Гц)

def generate_tone(duration, frequency, sample_rate):
    """Генерирует тональный сигнал заданной длительности и частоты."""
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    tone = np.sin(frequency * t * 2 * np.pi)
    return tone

def generate_silence(duration, sample_rate):
    """Генерирует тишину заданной длительности."""
    return np.zeros(int(sample_rate * duration))

def morse_code(message):
    """Преобразует сообщение в последовательность сигналов азбуки Морзе."""
    morse_dict = {
        'S': '...',
        'O': '---',
    }

    signals = []
    for char in message:
        if char.upper() in morse_dict:
            code = morse_dict[char.upper()]
            for symbol in code:
                if symbol == '.':
                    signals.append(generate_tone(dot_duration, frequency, sample_rate))
                elif symbol == '-':
                    signals.append(generate_tone(dash_duration, frequency, sample_rate))
                signals.append(generate_silence(pause_duration, sample_rate))
            signals.append(generate_silence(word_pause, sample_rate))
    return np.concatenate(signals)

# Генерация сигнала SOS
sos_signal = morse_code("SOS")

In [80]:
# creating the classes
from pynput import keyboard as kb
from IPython.display import Image, display
import time





class HauntedMansion:
    def __init__(self):
        self.rooms = {}
        self.doors = {}

    def add_room(self, *rooms):
        for room in rooms:
            self.rooms[room.name] = room

    def add_door(self, *doors):
        for door in doors:
            if door.room1.name in self.rooms and door.room2.name in self.rooms:
                self.doors[door.name] = door
                door.room1.add_door(door)
                door.room2.add_door(door)

class Room:
    def __init__(self, name, description, image=None) :
        self.name = name
        self.description = description
        self.objects = {}
        self.doors = []
        self.image = image

    def add_door(self, door):
         if door not in self.doors:
             self.doors.append(door)

    def add_object(self, *objs):
        for obj in objs:
            self.objects[obj.name] = obj

    def search(self):
        display(Image(filename=self.image))
        if self.objects:
            print("You found:")
            for obj in self.objects.values():
                print(f"- {obj.name}: {obj.description}")
        else:
            print("There's nothing interesting here.")
        if self.doors:
             if len(self.doors) == 1:
                 print("You also find that there is a door:")
             else:
                 print(f"You also find that there are {len(self.doors)} doors:")
             for door in self.doors:
                print(f"- {door.name}")
        else:
             print("There are no doors here.")

'''
class MorseCode:
    def __init__(self):
        self.sample_rate = 44100
        self.dot_duration = 0.1
        self.dash_duration = 0.3
        self.pause_duration = 0.1
        self.word_pause = 0.5
        self.frequency = 440

    @staticmethod
    def generate_tone(duration, frequency, sample_rate):
        """Генерирует тональный сигнал заданной длительности и частоты."""
        t = np.linspace(0, duration, int(sample_rate * duration), False)
        tone = np.sin(frequency * t * 2 * np.pi)
        return tone

    @staticmethod
    def generate_silence(duration, sample_rate):
        """Генерирует тишину заданной длительности."""
        return np.zeros(int(sample_rate * duration))

    def morse_code(self, message):
        """Преобразует сообщение в последовательность сигналов азбуки Морзе."""
        morse_dict = {
            'S': '...',
            'O': '---',
        }

        signals = []
        for char in message:
            if char.upper() in morse_dict:
                code = morse_dict[char.upper()]
                for symbol in code:
                    if symbol == '.':
                        signals.append(self.generate_tone(self.dot_duration, self.frequency, self.sample_rate))
                    elif symbol == '-':
                        signals.append(self.generate_tone(self.dash_duration, self.frequency, self.sample_rate))
                    signals.append(self.generate_silence(self.pause_duration, self.sample_rate))
                signals.append(self.generate_silence(self.word_pause, self.sample_rate))
        return np.concatenate(signals)

        '''

class Object:
    def __init__(self, name, description, item=None, key_required=None, interaction_message=None):
        self.name = name
        self.description = description
        self.item = item
        self.key_required = key_required
        self.interaction_message = interaction_message

    def interact(self, player):
        if self.item:
            if self.interaction_message:
                print(self.interaction_message)
            else:
                print(f"You found a {self.item.name} inside the {self.name}! {self.item.description}")
            player.inventory.append(self.item)
            self.item = None  # The item is taken
        else:
            print(f"The {self.name} is empty.")

class Chest(Object):
    def __init__(self, name, description, item = None, is_locked=True, correct_pin = None):
        super().__init__(name, description, item)
        self.is_locked = is_locked
        self.correct_pin = correct_pin 
    
    def interact(self, player):
        if self.is_locked:
            print(f"The {self.name} is locked, try to open it! Enter the 4-digit PIN code or type 'cancel':")
            while True:
                pin = input()
                if pin.lower() == "cancel":
                    print("You decided not to try the PIN code.")
                    return  
                try:
                    if len(pin) != 4 or not pin.isdigit():  # check length and if cod contains digits
                        raise ValueError
                    if int(pin) == self.correct_pin:  # check if the pin is correct
                        self.is_locked = False
                        print(f"You unlocked the {self.name}!")
                        super().interact(player)  # open the chest 
                        return  
                    else:
                        print("Incorrect PIN. Try again or type 'cancel':")
                except ValueError:
                    print("Invalid input. Please enter a 4-digit PIN or type 'cancel':")

        else: 
            print("It looks like someone already opened it. There's nothing interesting here.")


class Radio(Object):
    def __init__(self, name, description, item=None, is_locked=True, correct_code="SOS"):
        super().__init__(name, description, item)
        self.is_locked = is_locked
        self.correct_code = correct_code  
        self.morse = MorseCode()  

    def interact(self, player):
        
        print(f"The {self.name} is locked. You hear strange sounds from inside ... --- ...")
        Audio(sos_signal, rate=sample_rate)
      
        
        if self.is_locked:

            print("The sounds stopped. You can now try to unlock the radio.")
            print("Enter the code you heard or type 'EXT':")

            while True:
                code = input().upper()
                if code.upper() == "EXT":
                    print("You decided not to try the code.")
                    return
                if code.upper() == self.correct_code:
                    self.is_locked = False
                    print(f"You unlocked the {self.name}!")
                    super().interact(player)
                    return
                else:
                    print("Incorrect code. Try again or type 'EXT':")
        else:
            print("It looks like someone already opened it. There's nothing interesting here.")

class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

class Door:
    def __init__(self, name, room1, room2, key_required = None):
        self.name = name
        self.room1 = room1
        self.room2 = room2
        self.key_required = key_required

    def get_other_room(self, current_room):
        return self.room2 if current_room == self.room1 else self.room1

class Player:
    def __init__(self):
        self.current_room = None
        self.inventory = []
        self.current_action = None
        self.running = True
        self.all_actions = []


    def game_loop(self):
        instructions = "Use 'W' to move, 'Q' to search, 'E' to interact, 'I' to list inventory, 'Esc' to quit."
        further_instructions = "To move through a door or interact with an object, please press the corresponding number of your choice."

        print(f"\nYou are in {self.current_room.name}. {self.current_room.description}")
        print(instructions + " " + further_instructions)
        print("Use 'H' to display instructions again at any point.")
        self.search_room()  # Sergej:  Search the room

        def on_press(key):
            try:
                if self.current_action is None:
                    if key.char == "h":  # Show instructions
                        print("\n--- Instructions ---")
                        print(instructions)
                        print("--------------------")
                    elif key.char == "w":
                        self.move()
                    elif key.char == "q":
                        self.search_room()
                    elif key.char == "e":
                        self.interact_with_object()
                    elif key.char == "i":
                        self.check_inventory()

                elif self.current_action == "moving" and key.char.isdigit():
                    self.handle_move(int(key.char))

                elif self.current_action == "interacting" and key.char.isdigit():
                    self.handle_interaction(int(key.char))

            except AttributeError:
                if key == kb.Key.esc:
                    if self.current_action is None:
                        print("Exiting game...")
                        self.running = False  # Stop the game loop
                        return False  # Stops listener
                    else:
                        print("Exiting action...")
                        self.current_action = None

        with kb.Listener(on_press=on_press) as listener:
            while self.running:
                pass

        listener.stop()

        print("Game loop ended.")

    def move(self):
        if self.current_action:
            return
        
        self.current_action = "moving"
        doors = self.current_room.doors
        if not doors:
            print("There are no doors here. How did you even get here?")
            self.current_action = None
            return
        
        print("\nWhere do you want to move?")
        for index, door in enumerate(doors, start=1):
            print(f"{index} - {door.name}")
        print("Press a number to select a door or 'Esc' to cancel.")


    def handle_move(self, index):
        if self.current_action != "moving":
            return
        
        doors = self.current_room.doors
        if 1 <= index <= len(doors):
            selected_door = doors[index - 1]
            other_room = selected_door.get_other_room(self.current_room)
            if selected_door.key_required and selected_door.key_required not in self.inventory:
                print(f"You need a {selected_door.key_required.name} to go there.")
            else:
                self.current_room = other_room
                print(f"You moved to {other_room.name}. {other_room.description}")

                if self.current_room.name == "Outside":
                    display(Image(filename=self.current_room.image))        #  Sergej added this line
                    self.all_actions.append([self.current_action, self.current_room.name])  # Sergej:  Add the action to the list of all actions
                    print(f"Congratulations! You've escaped the haunted mansion after {len(self.get_all_actions())} actions!")
                    #print(*self.get_all_actions(), sep = "\n")  # Sergej:  Print all actions
                    self.running = False  # Stop the game loop
                    return False
                else:
                    self.all_actions.append([self.current_action, self.current_room.name])  # Sergej:  Add the action to the list of all actions
                    self.search_room()                         # Sergej:  Search the new room

        self.current_action = None  # Exit moving mode
    

    def search_room(self):
        self.current_room.search()


    def interact_with_object(self):
        if self.current_action:
            return
        
        self.current_action = "interacting"
        objects = self.current_room.objects
        if not objects:
            print("There's nothing to interact with here.")
            self.current_action = None
            return

        print("\nWhat do you want to interact with?")
        for index, obj in enumerate(objects.values(), start=1):
            print(f"{index} - {obj.name}")
        print("Press a number to interact or 'Esc' to cancel.")
        

    def handle_interaction(self, index):
        if self.current_action != "interacting":
            return
        
        objects = self.current_room.objects
        if 1 <= index <= len(objects):
            obj = list(objects.values())[index - 1]
            if obj.key_required and obj.key_required not in self.inventory:
                print(f"You need another item to access this Object. Maybe look somewhere else?")
            else:
                obj.interact(self)
        
        self.all_actions.append([self.current_action, obj.name])   # Sergej:  Add the action to the list of all actions
        self.current_action = None  # Exit interaction mode

    def check_inventory(self):
        if self.inventory:
            print("\nInventory:")
            for item in self.inventory:
                print(f"- {item.name}: {item.description}")
        else:
            print("\nYour inventory is empty.")

    def get_all_actions(self):
        return self.all_actions

In [81]:
# creating & setting the game

# Create the House
house = HauntedMansion()


# Create Rooms
# Ground Floor
outside = Room("Outside", "A breeze of fresh air and singing birds.", "imgs/outdoor.png")
kitchen = Room("Kitchen", "A warm kitchen with a smell of fresh bread.", "imgs/kitchen.png")
dining_room = Room("Dining Room", "A nicely decorated room with a big table to have a meal.", "imgs/dining.png")
living_room = Room("Living Room", "A huge living room. You hear some songs from the radio.", "imgs/living_room.png")
wc = Room("WC", "A small room with a toilet and a sink.", "imgs/wc.png")
hallway_ground_floor = Room("Hallway (Ground Floor)", "An empty hallway on the ground floor.", "imgs/hallway.png")
# First Floor
master_bedroom = Room("Master Bedroom", "A peaceful big bedroom with a soft king-size bed.", "imgs/bedroom.png")
little_guest_room = Room("Little guest Room", "A little room for guests.", "imgs/little_guest_room.png")
big_guest_room = Room("Big guest Room", "A big room for guests.", "imgs/big_guest_room.png")
hallway_first_floor = Room("The hallway with doors", "An empty hallway with stairs that lead downstairs.", "imgs/hallway1.png")
small_bathroom = Room("Small Bathroom", "A bathroom for guests with a bathtub and a shower.", "imgs/bath.png")
master_bathroom = Room("Master Bathroom", "A big cold bathroom with a toilet and a sink.", "imgs/bathroom.png")
# Keys
silver_key = Item("Silver key", "It opens the silver door")
golden_key = Item("Golden key", "It opens the golden door")
bronze_key = Item("Bronze key", "It opens the bronze door")
platin_key = Item("Platinum key", "It opens the platinum door")
master_key = Item("Master key", "It opens the big door to the outside")
# Items
hammer = Item("Hammer", "Now you can break something!")
chest_code_1 = Item("A piece of paper", "It contains some strange symbols: 'Код от комода: 19 ...'. Than  the paper is torn.")
chest_code_2 = Item("A tiny piece of paper", "There are just two numbers on it: '20.'. It seems to be a part of a bigger paper.")


# Add Rooms to the House
house.add_room(kitchen, hallway_ground_floor, master_bedroom, dining_room, outside, living_room, wc,
        little_guest_room, big_guest_room, hallway_first_floor, small_bathroom, master_bathroom)


# Create Doors
# Ground Floor
silver_door = Door("Silver door", dining_room, kitchen, silver_key)
white_door = Door("White door", dining_room, hallway_ground_floor)
gold_door = Door("Golden door", hallway_ground_floor, living_room, golden_key)
big_door = Door("Big door", hallway_ground_floor, outside, master_key)
green_door = Door("Green door", hallway_ground_floor, wc)
stairs = Door("Stairs", hallway_ground_floor, hallway_first_floor)
# First Floor
bronze_door = Door("Bronze door", hallway_first_floor, little_guest_room, bronze_key)
platin_door = Door("Platinum door", hallway_first_floor, master_bedroom, platin_key)
black_door = Door("Black door", hallway_first_floor, small_bathroom)
red_door = Door("Red door", hallway_first_floor, big_guest_room)
yellow_door = Door("Yellow door", master_bedroom, master_bathroom)


# Connect rooms with doors
house.add_door(silver_door, white_door, gold_door, big_door, green_door, stairs,
        bronze_door, platin_door, black_door, red_door, yellow_door)


# Create Objects
# Ground Floor
chest = Chest("Chest", "A mysterious chest.", golden_key, is_locked=True, correct_pin = 1920)
cupboard = Object("Cupboard", "A brown wooden cupboard.", chest_code_2)
drawer = Object("Drawer", "An old wooden drawer.", chest_code_1)
cabinet = Object("Cabinet", "A huge black cabinet.")
radio = Radio("Radio", "An ancient radio.", master_key, is_locked=True, correct_code="SOS")
# First Floor
mirror = Object("Mirror", "A big dirty mirror with a wide frame.", platin_key, hammer, interaction_message="You've destroyed the mirror and found a platinum key in between the shards!")
little_drawer = Object("Little drawer", "A little drawer.", hammer)
desk = Object("Desk", "A wooden desk.", bronze_key)
bed = Object("Bed", "A messy bed. You woke up here!")
king_bed = Object("King bed", "A huge, dusty bed with unkempt sheets.", silver_key, interaction_message="You found a silver key under the pillow!")
safe = Object("Safe", "A safe with a digital lock.")


# Add Objcts to rooms
dining_room.add_object(drawer)
kitchen.add_object(chest, cupboard)
little_guest_room.add_object(little_drawer, mirror)
big_guest_room.add_object(bed, desk)
master_bedroom.add_object(king_bed, safe)
living_room.add_object(radio, cabinet)


# Create a player and set their starting room
player = Player()
player.current_room = big_guest_room

In [ ]:
start_time = time.time()

player.game_loop()

end_time = time.time()
elapsed_time = end_time - start_time
minutes = int(elapsed_time // 60) 
seconds = int(elapsed_time % 60) 

print(f"Time taken to complete the game: {minutes} minutes and {seconds} seconds .")
